In [1]:
import pandas as pd
import numpy as np
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score
import os

In [4]:
def to_adjascency_mat(edges, numOfNodes):
    
    m = np.zeros((numOfNodes, numOfNodes))
    
    for e in edges:
        i, j = e[0], e[1]
        m[i-1,j-1] = 1
    return m

# Crate target filename and Load data
#dataFile = '../Data/Sachs/1. cd3cd28.xls'
origFolder = 'Synthetic_data'
genProc = 'gaussian_different_noise_var'
dataFile = os.path.join('..', 'Data', origFolder, genProc, 'data.npy')
fileType = dataFile.split('.')[-1]
# Use appropriate laoding procedure according to file type
if fileType == 'xls':
    df = pd.read_excel(dataFile)
elif fileType == 'npy' or fileType == 'txt':
    data = np.load(dataFile)
    df = pd.DataFrame(data).astype(np.float32)

# Construct columns and normalize them for convergence speed
df.columns = [i for i in range(0, df.shape[1])] # REname columns to numbers for easy of use and readability
normalized_df=((df-df.min())/(df.max()-df.min())).astype(np.float32)
#print(normalized_df, df.max())
print(df)

            0          1         2          3         4          5   \
0     0.741812  -3.520448  3.836546   1.200599  0.045833  -0.908260   
1     1.626519   6.017479 -1.368640   1.334406 -0.369819   3.118125   
2     0.974898 -13.754857  3.593695   0.339380  0.743079  -1.763826   
3     1.307865  12.938993 -0.174701  -0.655204 -1.908801  14.835423   
4     0.329775   9.131368  2.120937 -10.812152 -3.191463  22.972898   
...        ...        ...       ...        ...       ...        ...   
4995 -1.849562 -11.790884 -1.106104   1.700977  2.090267 -15.613702   
4996  1.076216  -0.519650  2.036813  -1.742626 -0.549174   6.660651   
4997  0.743386  -2.394277 -0.689815   9.552317  1.455592 -14.995937   
4998  0.034964  -3.580285  1.509398  -5.969070 -0.638323   2.750664   
4999 -1.207876 -12.461382 -0.332142   0.086688  2.392320 -14.722924   

             6         7         8         9         10        11  
0      0.800373 -0.573631  0.405245 -0.700352 -0.219791 -2.675076  
1      1.24

In [22]:
# create data sample with 9 random variables:
data = pd.DataFrame(np.random.randint(0, 5, size=(5000, 9)), columns=list('ABCDEFGHI'))

# add 10th dependent variable
data['J'] = data['A'] * data['B']
print(df)
est = HillClimbSearch(data, scoring_method=BicScore(data))
best_model = est.estimate()
print(sorted(best_model.nodes()))
print(best_model.edges())
j = list(best_model.edges())
# search a model with restriction on the number of parents:
# est.estimate(max_indegree=1).edges()




  0%|                                                                                      | 0/1000000 [00:00<?, ?it/s]

        0     1      2       3      4      5      6      7      8     9   \
0     26.4  13.2   8.82   18.30  58.80   6.61  17.00  414.0  17.00  44.9   
1     35.9  16.5  12.30   16.80   8.13  18.60  32.50  352.0   3.37  16.5   
2     59.4  44.1  14.60   10.20  13.00  14.90  32.50  403.0  11.40  31.9   
3     73.0  82.8  23.10   13.50   1.29   5.83  11.80  528.0  13.70  28.6   
4     33.7  19.8   5.19    9.73  24.80  21.10  46.10  305.0   4.66  25.7   
..     ...   ...    ...     ...    ...    ...    ...    ...    ...   ...   
848   42.2  27.6  21.50   67.90  12.30   8.28  15.80  562.0  21.90  29.7   
849   62.1  21.7   9.56    6.49  34.30  15.30  23.10  319.0  11.90  35.5   
850  103.0  32.2  10.20   13.20  50.90   5.28   5.62  239.0  14.50  22.9   
851   77.7  23.7  24.80   60.40  44.50  21.50  25.00  289.0  32.20  83.5   
852   46.6  15.0   8.51  120.00  22.30   6.10  20.00  478.0  20.90  36.8   

        10  
0    40.00  
1    61.50  
2    19.50  
3    23.10  
4    81.30  
..     ..



  0%|                                                                          | 1/1000000 [00:00<264:53:01,  1.05it/s]

  0%|                                                                          | 2/1000000 [00:01<195:10:04,  1.42it/s]

  0%|                                                                          | 3/1000000 [00:01<122:49:51,  2.26it/s]

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
[('B', 'A'), ('J', 'B'), ('J', 'A')]


In [5]:
est = HillClimbSearch(df, scoring_method=BicScore(df))
best_model = est.estimate( max_iter = 50)

  2%|█▌                                                                              | 1/50 [05:41<4:39:00, 341.65s/it]

MemoryError: Unable to allocate 931. GiB for an array with shape (24995000, 5000) and data type float64

In [8]:
print(best_model.edges())
m = list(best_model)

[(1, 0), (5, 6), (6, 9), (7, 5), (7, 2), (8, 10), (9, 3), (9, 8), (10, 4)]


In [9]:
m = to_adjascency_mat(best_model.edges(), 11)
#saveFileBase = './Results/mmhc_sachs_data_1'
saveFileBase = './Results
saveFile = os.path.join(saveFileBase, '_'.join(('mmhc', genProc, 'predG')))
np.save(saveFileBase+'.npy', m)
np.savetxt(saveFileBase, m)
print(m)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
from rpy2.robjects import default_converter
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# use the default conversion rules to which the pandas conversion
# is added
with localconverter(default_converter + pandas2ri.converter) as cv:
    dataf = robjects.r["data"]